## Aufgabe zum Gradientenverfahren

In [236]:
# in this code block we gather all imports

import numpy as np

In [237]:
# in this code block weight values are initialized in a very explicit verbose fashion

# this is the number of weights in our model function
# so it is also going to be the number of values in our vector of weights w
number_of_weights = 2

# in the following the variables prefixed with w_pre_ are just intermediate variables
# the variable w_init is our vector of weights with their initial values

# we draw some random values based on a normal distribution
# we actually draw two more since during the below min-max-normalization
# one value becomes .0 and another value might become 1.
w_pre_number_of_values = number_of_weights + 2
w_pre_random_values = np.random.normal(loc=.5, scale=.2, size=w_pre_number_of_values)
print(f"w_pre={w_pre_random_values}")

# the values in w_pre_initialized can be below 0 and above 1 so we min-max-normalize them
# just out of curiosity through an explicit inline computation based on numpy primitives
# formula is value_min_max_normalized = value - min / max - min

w_pre_min = w_pre_random_values.min()
assert w_pre_min.shape == ()
print(f"w_pre_min={w_pre_min}")

w_pre_max = w_pre_random_values.max()
assert w_pre_max.shape == ()
print(f"w_pre_max={w_pre_max}")

w_pre_range = w_pre_max - w_pre_min
assert w_pre_range.shape == ()
print(f"w_pre_range={w_pre_range}")

w_pre_range_reciprocal = 1 / w_pre_range
assert w_pre_range_reciprocal.shape == ()
print(f"w_pre_range_reciprocal={w_pre_range_reciprocal}")

w_pre_min_repeated = np.full_like(w_pre_random_values, w_pre_min)
assert w_pre_min_repeated.shape == (w_pre_number_of_values,)
print(f"w_pre_min_repeated={w_pre_min_repeated}")

w_pre_minus_min = np.subtract(w_pre_random_values, w_pre_min_repeated)
assert w_pre_minus_min.shape == (w_pre_number_of_values,)
print(f"w_pre_minus_min={w_pre_minus_min}")

w_pre_min_max_normalized = w_pre_minus_min * w_pre_range_reciprocal
assert w_pre_min_max_normalized.shape == (w_pre_number_of_values,)
print(f"w_pre_min_max_normalized={w_pre_min_max_normalized}")

# we now mask the values that are equal or very close to the minimum .0 or the maximum .1
w_pre_masked = np.ma.masked_outside(w_pre_min_max_normalized, .0001, .9999, copy=True)
assert w_pre_masked.shape == (w_pre_number_of_values,)
print(f"w_pre_masked={w_pre_masked}")

# to finally drop them and thus obtain our weights
w_init = w_pre_masked.compressed()
print(f"w_init={w_init}")
assert w_init.shape == (number_of_weights,)


w_pre=[0.64728919 0.46650823 0.42658773 0.6567061 ]
w_pre_min=0.42658773342947237
w_pre_max=0.6567060989890093
w_pre_range=0.23011836555953696
w_pre_range_reciprocal=4.3455897036661195
w_pre_min_repeated=[0.42658773 0.42658773 0.42658773 0.42658773]
w_pre_minus_min=[0.22070145 0.0399205  0.         0.23011837]
w_pre_min_max_normalized=[0.95907797 0.1734781  0.         1.        ]
w_pre_masked=[0.9590779671323574 0.17347809650336948 -- --]
w_init=[0.95907797 0.1734781 ]
